## Foxes and Bunnies Simulation

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import math 

class BunnyFoxesSimulation:
    """
    Manages the 2d cellular automaton for a predator-prey simulation.
    """
    def __init__(self, n=200, b_cap=100.0, b_growth=0.3, b_diff=0.1, 
                 f_consump=0.5, f_growth=0.1, f_mort=0.05, f_diff=0.1,
                 p_bunny=0.5, p_fox=0.1, seed=42):
        """
        Initializes the simulation parameters.

        args:
            n (int): grid dimension (n x n).
            b_cap (float): bunny carrying capacity per cell.
            b_growth (float): bunny reproduction rate.
            b_diff (float): bunny diffusion rate.
            f_consump (float): fox consumption rate.
            f_growth (float): fox reproduction efficiency.
            f_mort (float): fox mortality rate.
            f_diff (float): fox diffusion rate.
            p_bunny (float): initial bunny population percentage (0.0 to 1.0).
            p_fox (float): initial fox population percentage (relative to bunnies).
            seed (int): seed for random number generators.
        """
        self.n = n
        self.b_cap = float(b_cap)
        self.b_growth = float(b_growth)
        self.b_diff = float(b_diff)
        self.f_consump = float(f_consump)
        self.f_growth = float(f_growth)
        self.f_mort = float(f_mort)
        self.f_diff = float(f_diff)
        self.p_bunny = p_bunny
        self.p_fox = p_fox
        self.seed = seed
        
        self.bunnies = np.zeros((self.n, self.n))
        self.foxes = np.zeros((self.n, self.n))
        
    def initialize(self):
        """
        Populates the bunny and fox grids with initial random densities.
        """
        rng = np.random.RandomState(self.seed)
        self.bunnies = rng.uniform(0, self.b_cap * self.p_bunny, size=(self.n, self.n))
        if self.p_fox > 0:
            self.foxes = rng.uniform(0, self.b_cap * self.p_bunny * self.p_fox, size=(self.n, self.n))
        else:
            self.foxes.fill(0)

    def update(self):
        """
        Advances the simulation by one time step using the assignment rules
        using numpy for increased efficiency.
        """
        # bunny reproduction
        b_repro_change = self.b_growth * self.bunnies * (1 - self.bunnies / self.b_cap)
        b_available = self.bunnies + b_repro_change
        
        # fox hunting
        b_hunted = np.minimum(b_available, self.f_consump * self.foxes)
        
        # fox reproduction and starvation
        surviving_foxes = self.foxes * (b_hunted / (self.f_consump * self.foxes + 1e-6))
        f_repro_change = self.f_growth * surviving_foxes
        f_starve_change = surviving_foxes - self.foxes
        f_available = self.foxes + f_starve_change + f_repro_change
        
        # calculate sum of neighbors for diffusion
        b_neighbor_sum = (np.roll(b_available, 1, axis=0) + 
                          np.roll(b_available, -1, axis=0) + 
                          np.roll(b_available, 1, axis=1) + 
                          np.roll(b_available, -1, axis=1))
        
        f_neighbor_sum = (np.roll(f_available, 1, axis=0) + 
                          np.roll(f_available, -1, axis=0) + 
                          np.roll(f_available, 1, axis=1) + 
                          np.roll(f_available, -1, axis=1))

        # bunny diffusion
        b_diff_change = self.b_diff * ((b_neighbor_sum / 4.0) - b_available)
        
        # fox diffusion
        f_diff_change = self.f_diff * ((f_neighbor_sum / 4.0) - f_available)
        
        # apply deltas
        bunny_delta = b_repro_change - b_hunted + b_diff_change
        fox_delta = f_starve_change + f_repro_change + f_diff_change
        
        # fox catastrophic mortality event
        mortality_event = np.random.rand(self.n, self.n) < self.f_mort
        fox_delta[mortality_event] = -self.foxes[mortality_event] 
        
        # apply changes
        self.bunnies += bunny_delta
        self.foxes += fox_delta
        
        # clip populations to valid ranges
        np.clip(self.bunnies, 0, self.b_cap, out=self.bunnies)
        np.clip(self.foxes, 0, None, out=self.foxes)

## Test Cases

In [ ]:
import numpy as np
import numpy.testing as npt

# numpy.testing was used here to quickly compare if elements 
# in the entire array align with expectations

def test_bunny_growth(g_b, k_b, b_initial):
    """
    Tests bunny logistic growth in isolation by
    turning off all diffusion and fox parameters, then check
    if a uniform grid of bunnies grows according to the
    logistic equation: d_b = b_growth * b * (1 - b / b_cap)

    args:
        g_b (float): bunny reproduction rate.
        k_b (float): bunny carrying capacity per cell.
        b_initial (float): uniform initial bunny density.
    """
    n = 20
    
    # initialize simulation with diffusion and foxes disabled
    sim = BunnyFoxesSimulation(
        n=n, 
        b_cap=k_b,
        b_growth=g_b,
        b_diff=0.0,
        f_consump=0.0, # no hunting
        f_growth=0.0,  # no fox growth
        f_mort=0.0,    # no fox death
        f_diff=0.0
    )
    
    # set uniform initial populations
    sim.bunnies = np.full((n, n), b_initial)
    sim.foxes = np.zeros((n, n))
    
    # calculate expected change
    # growth = 0.1 * 50 * (1 - 50 / 100) = 5 * (0.5) = 2.5
    expected_growth = g_b * b_initial * (1 - b_initial / k_b)
    expected_bunnies = np.full((n, n), b_initial + expected_growth)
    
    # run one step
    sim.update()
    
    # assert
    npt.assert_allclose(
        sim.bunnies, 
        expected_bunnies, 
        rtol=0,
        err_msg=f"Bunny growth calculation is incorrect. g_b={g_b}, k_b={k_b}"
    )
    npt.assert_allclose(
        sim.foxes, 
        np.zeros((n, n)), 
        atol=0, # use atol for checking zeros
        err_msg="Foxes appeared unexpectedly."
    )

def test_fox_starvation(f_initial, f_consump, f_growth):
    """
    Tests fox starvation logic in isolation by reducing
    the bunny growth rate and checking if the foxes die out.
    
    args:
        f_initial (float): uniform initial fox density.
        f_consump (float): fox consumption rate.
        f_growth (float): fox reproduction efficiency.
    """
    n = 20
    # f_initial = 40.0 # now a parameter
    
    # initialize simulation with diffusion and bunnies disabled
    sim = BunnyFoxesSimulation(
        n=n, 
        b_cap=100.0,
        b_growth=0.0,
        b_diff=0.0,
        f_consump=f_consump,
        f_growth=f_growth,  
        f_mort=0.0,    # turn off to prevent random events
        f_diff=0.0
    )
    
    # set uniform initial populations
    sim.bunnies = np.zeros((n, n)) # no bunnies
    sim.foxes = np.full((n, n), f_initial)
    
    # calculate expected change
    # b_hunted = 0
    # surviving_foxes = 0
    # f_repro_change = 0
    # f_starve_change = 0 - f_initial = -40.0
    # fox_delta = -40.0 + 0 + 0 = -40.0
    # self.foxes = 40.0 - 40.0 = 0.0
    expected_foxes = np.zeros((n, n))
    
    # run one step
    sim.update()
    
    # assert
    npt.assert_allclose(
        sim.foxes, 
        expected_foxes, 
        atol=0, 
        err_msg=f"Fox starvation logic is incorrect. f_initial={f_initial}"
    )

def test_bunny_diffusion(d_b, center_pop):
    """
    Tests bunny diffusion in isolation by turning off all growth/death parameters. 
    
    args:
        d_b (float): bunny diffusion rate.
        center_pop (float): population at the single center cell.
    """
    n = 50 
    
    # initialize simulation with growth/death disabled
    sim = BunnyFoxesSimulation(
        n=n, 
        b_cap=100.0,
        b_growth=0.0,
        b_diff=d_b,
        f_consump=0.0,
        f_growth=0.0,
        f_mort=0.0,
        f_diff=0.0
    )
    
    # set initial populations: one spike in the middle
    b_initial = np.zeros((n, n))
    center = n // 2
    b_initial[center, center] = center_pop
    
    sim.bunnies = b_initial.copy()
    sim.foxes = np.zeros((n, n))
    
    # calculate expected outcome
    # center cell:
    #   b_available = 100.0
    #   b_neighbor_sum = 0.0
    #   b_diff_change = 0.1 * ((0.0 / 4.0) - 100.0) = -10.0
    #   final = 100.0 - 10.0 = 90.0
    #
    # neighbor cell (e.g., center-1, center):
    #   b_available = 0.0
    #   b_neighbor_sum = 100.0 (from center cell)
    #   b_diff_change = 0.1 * ((100.0 / 4.0) - 0.0) = 0.1 * 25.0 = 2.5
    #   final = 0.0 + 2.5 = 2.5
    
    # dynamic calculation based on params
    final_center = center_pop + (d_b * (0.0 - center_pop))
    final_neighbor = 0.0 + (d_b * (center_pop / 4.0))

    expected_bunnies = np.zeros((n, n))
    expected_bunnies[center, center] = final_center
    expected_bunnies[center-1, center] = final_neighbor
    expected_bunnies[center+1, center] = final_neighbor
    expected_bunnies[center, center-1] = final_neighbor
    expected_bunnies[center, center+1] = final_neighbor
    
    # run one step
    sim.update()
    
    # assert
    npt.assert_allclose(
        sim.bunnies, 
        expected_bunnies, 
        atol=0,
        err_msg=f"Bunny diffusion calculation is incorrect. d_b={d_b}"
    )



def run_all_tests():
    """
    Finds and runs all functions in this file starting with 'test_'.
    """
    print("Running simulation tests...")
    test_failures = []
    
    # define all test cases to run
    test_cases = [
        (test_bunny_growth, (0.1, 100.0, 50.0), "test_bunny_growth (standard)"),
        (test_bunny_growth, (0.2, 100.0, 20.0), "test_bunny_growth (low pop)"),
        (test_bunny_growth, (0.1, 100.0, 99.0), "test_bunny_growth (high pop)"),
        
        (test_fox_starvation, (40.0, 0.5, 0.1), "test_fox_starvation (standard)"),
        (test_fox_starvation, (10.0, 0.1, 0.1), "test_fox_starvation (low pop)"),

        (test_bunny_diffusion, (0.1, 100.0), "test_bunny_diffusion (standard)"),
        (test_bunny_diffusion, (0.5, 100.0), "test_bunny_diffusion (high diffusion)"),
    ]

    tests_run = len(test_cases)
    
    for func, args, name in test_cases:
        try:
            # call the function with its arguments
            func(*args)
            print(f"[SUCCESS] {name}")
        except Exception as e:
            # catches assertion errors from npt.assert_allclose
            print(f"[FAILURE] {name}\n    {e}")
            test_failures.append((name, e))

    print("-" * 40)
    if not test_failures:
        print(f"All {tests_run} tests passed.")
    else:
        print(f"{len(test_failures)} / {tests_run} test(s) failed. See output above.")
    print("-" * 40)


run_all_tests()

### Convergence Check Functions

In [ ]:
def convergence_check(sim_plot_points, no_intervals=10, threshold=0.5, num_recent_changes_to_avg=3):
    '''
    Checks if the simulation has converged based on the change in means
    between time intervals. Uses a specified threshold.

    args:
    sim_plot_points (list/np.array): list of data points (e.g., mean population) over time.
    no_intervals (int): number of intervals to divide the data into for averaging.
    threshold (float): percentage change threshold to declare convergence.
    num_recent_changes_to_avg (int): number of recent interval changes to average for the check.

    returns:
        interval_ends (list): list of integer step indices for interval boundaries.
        avg_per_interval (list): list of floats, one for each interval's average value.
        perc_change_list (list): list of floats for the percentage change between intervals.
        has_converged_bool (bool): True if convergence threshold is met, False otherwise.
        avg_recent_change (float): the average absolute percentage change over recent intervals.
    '''
    steps = len(sim_plot_points)
    min_required_steps_for_intervals = no_intervals * 2 
    min_required_steps_for_check = (num_recent_changes_to_avg + 1) * (steps // no_intervals if no_intervals > 0 else 1)
    
    if steps < min_required_steps_for_intervals or steps < min_required_steps_for_check:
        return [], [], [], False, np.inf 

    if no_intervals <= 0: return [], [], [], False, np.inf
    interval_width = steps / no_intervals
    if interval_width < 1: return [], [], [], False, np.inf # interval too small

    interval_idx = []
    for i in range(1, no_intervals + 1):
        interval_idx.append(int(i * interval_width))

    avg_list = []
    current_idx_in_list = 0
    for chunk_num, int_upper_bound in enumerate(interval_idx):
        start_idx = current_idx_in_list
        end_idx = int_upper_bound if chunk_num < no_intervals - 1 else steps 
        chunk_data = sim_plot_points[start_idx:end_idx]
        if len(chunk_data) > 0: avg_list.append(np.mean(chunk_data))
        else: avg_list.append(avg_list[-1] if avg_list else 0) 
        current_idx_in_list = end_idx

    perc_list = []
    for i in range(1, len(avg_list)): 
        if avg_list[i-1] != 0:
            delta_perc = (avg_list[i] - avg_list[i-1]) / avg_list[i-1] * 100
            perc_list.append(delta_perc)
        else:
            perc_list.append(0.0 if avg_list[i] in [0, 0.0] else np.inf) 

    conv_bool = False
    avg_recent_abs_change = np.inf 

    if len(perc_list) >= num_recent_changes_to_avg:
        avg_recent_abs_change = np.mean(np.abs(perc_list[-num_recent_changes_to_avg:]))
        if avg_recent_abs_change <= threshold:
            conv_bool = True
            
    return interval_idx, avg_list, perc_list, conv_bool, avg_recent_abs_change



def get_convergence_times(sim_params, num_repeats=5, convergence_threshold=0.5, 
                          max_steps=10000, check_interval=200, 
                          num_intervals_for_check=10, progress_bar=True):
    """
    runs the simulation 'num_repeats' times until convergence or max_steps.
    returns a list containing the number of steps taken for each repeat.

    args:
    sim_params (dict): dictionary of parameters for the BunnyFoxesSimulation.
    num_repeats (int): number of times to repeat the simulation run.
    convergence_threshold (float): threshold passed to convergence_check.
    max_steps (int): maximum number of steps to run a simulation before stopping.
    check_interval (int): how often (in steps) to check for convergence.
    num_intervals_for_check (int): number of intervals for the convergence_check function.
    progress_bar (bool): whether to display a text-based progress bar.

    returns:
        steps_taken_list (list): list of integers, where each is the step count for a single run.
    """
    steps_taken_list = []
    
    if progress_bar:
        print(f"running {num_repeats} repeats to collect convergence times...")
        print(f"\rProgress: |{'-' * num_repeats}| 0/{num_repeats}", end="")

    for r in range(num_repeats):
        sim = BunnyFoxesSimulation(**sim_params) 
        sim.initialize()
        
        bunny_history = []
        converged = False
        step_count = 0
        
        while step_count < max_steps:
            sim.update()
            step_count += 1
            bunny_history.append(np.mean(sim.bunnies))
            
            min_steps_needed = num_intervals_for_check * 2 
            if step_count >= min_steps_needed and step_count % check_interval == 0:
                _, _, _, conv_bool, _ = convergence_check(
                    bunny_history, no_intervals=num_intervals_for_check, threshold=convergence_threshold
                )
                if conv_bool:
                    converged = True
                    break 
                    
        steps_taken_list.append(step_count) # store steps even if max_steps was hit

        if progress_bar:
            filled = (r + 1)
            bar = "█" * filled + "-" * (num_repeats - filled)
            print(f"\rProgress: |{bar}| {filled}/{num_repeats}", end="")

    if progress_bar:
        print()
        
    return steps_taken_list


def plot_convergence_time_histogram(convergence_times, sim_params):
    """
    plots a histogram of the convergence times collected from multiple runs.

    args:
    convergence_times (list/np.array): list of step counts, one for each simulation run.
    sim_params (dict): simulation parameters, used to get 'n' for the plot title.

    returns:
        None
    """
    plt.figure(figsize=(10, 6))
    
    # plot the histogram
    plt.hist(convergence_times, bins=20, color='khaki', edgecolor='darkgoldenrod')
    
    # calculate and plot mean/median lines
    mean_time = np.mean(convergence_times)
    median_time = np.median(convergence_times)
    plt.axvline(mean_time, color='red', linestyle='dashed', linewidth=1.5, label=f'Mean: {mean_time:.0f}')
    plt.axvline(median_time, color='purple', linestyle='dotted', linewidth=1.5, label=f'Median: {median_time:.0f}')
    
    plt.title(f'Distribution of Convergence Times (N={len(convergence_times)} runs, grid size={sim_params["n"]}x{sim_params["n"]})')
    plt.xlabel('Steps to Convergence', fontsize=14)
    plt.ylabel('Number of Runs', fontsize=14)
    plt.legend(fontsize=14)
    plt.grid(axis='y', alpha=0.2)
    plt.show()

In [ ]:
# histogram of convergence times
small_grid_params = {
    'n': 150,
    'b_growth': 0.3, 'b_diff': 0.01, 'f_consump': 0.3,
    'f_growth': 0.1, 'f_mort': 0.03, 'f_diff': 0.1, 
    'p_bunny': 0.3, 'p_fox': 0.1
}

convergence_times_list = get_convergence_times(
    small_grid_params, 
    num_repeats=400, 
    convergence_threshold=0.1, 
    max_steps=20000,
    check_interval=200, 
    num_intervals_for_check=10, 
    progress_bar=True
)

# plot histogram
plot_convergence_time_histogram(convergence_times_list, small_grid_params)


### Exploratory Variable Comparison Plot

In [ ]:
import pprint
def plot_variable_comparison(variable_to_test, values_to_test, steps, progress_bar=True):
    """
    Runs the simulation for different values of a chosen variable and plots the results
    in a grid of subplots for a fixed number of steps.

    args:
        variable_to_test (str): the name of the parameter in sim_params to vary (e.g., 'b_growth').
        values_to_test (list): a list of values to test for the specified variable.
        steps (int): the number of steps to run each simulation for.
        progress_bar (bool): whether to display a text-based progress bar.

    returns:
        None
    """
    print(f"Testing {variable_to_test}:")

    # creating a progressbar
    num_values = len(values_to_test)


    # setting up the layout of the plots
    cols = math.ceil(math.sqrt(num_values))
    rows = math.ceil(num_values / cols)
    fig, axes = plt.subplots(rows, cols, figsize=(cols * 5, rows * 4), 
                             sharex=True, sharey=False)
    
    if not isinstance(axes, (np.ndarray)):
        flat_axes = [axes]
    else:
        flat_axes = axes.flatten()

    pretty_names = {
        'b_growth': '$g_b$', 'b_diff': '$d_b$', 'f_consump': '$c_f$',
        'f_growth': '$g_f$', 'f_mort': '$m_f$', 'f_diff': '$d_f$'
    }
    var_name_pretty = pretty_names.get(variable_to_test, variable_to_test)

    base_params = {
        'n': 200, 'b_growth': 0.3, 'b_diff': 0.1, 'f_consump': 0.3,
        'f_growth': 0.1, 'f_mort': 0.02, 'f_diff': 0.1, 
        'p_bunny': 0.3, 'p_fox': 0.1
    }

    for i, p in enumerate(values_to_test):
        ax = flat_axes[i]
        sim_params = base_params.copy()
        sim_params[variable_to_test] = p
        
        # print out the other values of the simulation for better overview
        if i == 0:
            sim_params_print = sim_params.copy()
            sim_params_print.pop(variable_to_test)
            print("\t")
            pprint.pprint(sim_params_print)

        sim = BunnyFoxesSimulation(**sim_params)
        sim.initialize()
        
        b_density_history = np.zeros(steps)
        f_density_history = np.zeros(steps)
        
        for j in range(steps):
            b_density_history[j] = np.mean(sim.bunnies)
            f_density_history[j] = np.mean(sim.foxes)
            sim.update()
        
        ax.plot(b_density_history, label='bunnies', color='green', linewidth=1.5)
        ax.plot(f_density_history, label='foxes', color='sienna', linewidth=1.5, linestyle='--')
        
        if variable_to_test == 'b_diff': 
            rem = 'f_diff'
            ratio = lambda p, rem: round(p/base_params[rem], 2) if base_params[rem] != 0 else float('inf')
            ax.set_title(f'Population Dynamics with $d_b$/$d_f$ = {round(p,2)}/{base_params[rem]} = {ratio(p, rem)}', fontsize=12)
        elif variable_to_test == 'f_diff': 
            rem = 'b_diff'
            ratio = lambda p, rem: round(base_params[rem]/p, 2) if p != 0 else float('inf')
            ax.set_title(f'Population Dynamics with $d_b$/$d_f$ = {base_params[rem]}/{round(p,2)} = {ratio(p, rem)}', fontsize=12)
        else:
            ax.set_title(f'dynamics for {var_name_pretty} = {p:.2f}')
        ax.legend()
        ax.grid(alpha=0.2)
        ax.set_ylim(-10,110)
        
        
        if progress_bar:
            filled = (i + 1)
            bar = "█" * filled + "-" * (num_values - filled)
            print(f"\rProgress: |{bar}| {filled}/{num_values}", end="")
    
    if progress_bar:
        print() 
    
    for j in range(num_values, len(flat_axes)):
        flat_axes[j].axis('off')

    fig.suptitle(f'Population Dynamics vs. {var_name_pretty}', fontsize=16)
    fig.suptitle(f'Population Dynamics vs. {var_name_pretty}', fontsize=16)
    fig.supxlabel('Time Steps')
    fig.supylabel('Population Density')
    
    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
    plt.show()

In [ ]:
# plotting general fox growth rate for exploration
b_diff_values = [0.01,0.03,0.05,0.08, 0.1,0.3,0.5,0.6,0.7,0.8,0.9, 1]
plot_variable_comparison(variable_to_test='f_growth', values_to_test=b_diff_values, steps=3000)

In [ ]:
# plotting zoomed in fox growth rate
f_growth_values = np.arange(0, 1.1, 0.1)
plot_variable_comparison(variable_to_test='f_growth', values_to_test=f_growth_values, steps=3000)

## Threshold for fox population dying out

In [ ]:
# plotting a threshold where foxes always die out
def plot_viability_threshold_new(f_growth_values, num_repeats=10, progress_bar=True, debugging=False,
                                 convergence_threshold=0.01, 
                                 max_steps=10000, check_interval=200, 
                                 num_intervals_for_check=10):
    """
    Plots the final average fox population as a function of fox growth rate.
    Error bars represent the Standard Error of the Mean (SEM) across repeats.

    args:
        f_growth_values (list): a list of fox growth rates ($g_f$) to test.
        num_repeats (int): number of times to repeat the simulation for each $g_f$ value.
        progress_bar (bool): whether to display a text-based progress bar.
        debugging (bool): whether to print detailed status updates for each repeat.
        convergence_threshold (float): threshold passed to convergence_check.
        max_steps (int): maximum number of steps to run a simulation before stopping.
        check_interval (int): how often (in steps) to check for convergence.
        num_intervals_for_check (int): number of intervals for the convergence_check function.

    returns:
        final_avg_fox_populations (list): list of floats, the mean final fox density for each $g_f$ value.
    """
    
    # store the final averaged population density and its standard error
    final_avg_fox_populations = []
    sem_fox_populations = []
    converged_list = []
    
    num_values = len(f_growth_values)
    print(f"{num_values} parameter tests, with {num_repeats} repeats each.")
    if progress_bar:
        print(f"\rProgress: |{'-' * num_values}| 0/{num_values}", end="")

    # use base parameters
    base_params = {
        'n': 200, 'b_growth': 0.3, 'b_diff': 0.1, 'f_consump': 0.3,
        'f_growth': 0.1, 'f_mort': 0.03, 'f_diff': 0.1, 
        'p_bunny': 0.3, 'p_fox': 0.1
    }

    # for each value
    for i, p in enumerate(f_growth_values):
        
        repeat_final_densities = [] # store final density for each repeat
        
        # repeat the simulation for num_repeats times
        for r in range(num_repeats):
            sim_params = base_params.copy()
            sim_params['f_growth'] = p
            
            # set up the simulation
            sim = BunnyFoxesSimulation(**sim_params)
            sim.initialize()

            # set up parameters
            bunny_history = []
            fox_history = [] 
            converged = False
            step_count = 0
            avg_recent_abs_change = np.nan 
            
            while step_count < max_steps:
                sim.update()
                step_count += 1
                
                bunny_history.append(np.mean(sim.bunnies))
                fox_history.append(np.mean(sim.foxes))
                
                # check convergence
                min_steps_needed = num_intervals_for_check * 2 
                if step_count >= min_steps_needed and step_count % check_interval == 0:
                    _, _, _, conv_bool, avg_recent_abs_change = convergence_check(
                        bunny_history, # use bunny history for check
                        no_intervals=num_intervals_for_check, 
                        threshold=convergence_threshold
                    )
                    if conv_bool:
                        converged = True
                        break # exit while loop for this repeat
            
            converged_list.append(converged)
                        
            steps_to_average = min(len(fox_history), max(50, int(len(fox_history) * 0.1))) 
            
            if len(fox_history) > steps_to_average:
                 final_density = np.mean(fox_history[-steps_to_average:])
            elif len(fox_history) > 0:
                 final_density = np.mean(fox_history) # average whatever we have
            else:
                 final_density = 0.0 # if simulation didn't run at all

            repeat_final_densities.append(final_density)
            
            if debugging:
                status = "Converged" if converged else f"Stopped at max_steps ({max_steps})"
                print(f"\n  Param {p:.3f} | Repeat {r+1}/{num_repeats}: {status} after {step_count} steps with {round(avg_recent_abs_change, 4)}% change")

        # average the final densities from all repeats for this 'p' value
        final_avg_fox_populations.append(np.mean(repeat_final_densities))
        
        #
        if num_repeats > 1:
            std_dev = np.std(repeat_final_densities)
            std_err = std_dev / np.sqrt(num_repeats)
        else:
            std_err = 0.0 # no error if only a singel repeat
        sem_fox_populations.append(std_err)

        # update progress bar
        if progress_bar:
            filled = (i + 1)
            bar = "█" * filled + "-" * (num_values - filled)
            print(f"\rProgress: |{bar}| {filled}/{num_values}", end="")
    
    if progress_bar:
        print() 

    if debugging:
        print(f"Converged within {max_steps} steps: {converged_list}")
        
    plt.figure(figsize=(10, 6))

    plt.errorbar(
        f_growth_values,
        final_avg_fox_populations,
        yerr=sem_fox_populations,       
        fmt='o-',                       
        capsize=4,                      
        markersize=4,
        color='k',                      
        ecolor='gray',                  
        elinewidth=1,                   
        label='Mean $pm$ Standard Error'   
    )
    
    plt.xlabel('Fox Growth Rate ($g_f$)', fontsize=16)
    plt.ylabel('Final Average Fox Population', fontsize=16)
    plt.axhline(0, color='grey', linestyle='--', linewidth=0.8) 
    plt.grid(alpha=0.2)
    plt.xticks(fontsize=14)
    plt.yticks(fontsize=14)
    plt.legend()
    plt.show()

    return final_avg_fox_populations

In [ ]:
## general plot over the entire range to get a feeling
# i had to run the plots for the assignment overnight: the parameters are 100 points with 10 repetitions
f_growth_values = np.linspace(0, 1, 50)
num_repeats = 5
progress_bar = True
final_avg_fox_populations = plot_viability_threshold_new(f_growth_values, num_repeats, progress_bar, max_steps=12000, convergence_threshold=0.0)

In [ ]:
# extra-zoomed in plot
f_growth_values = np.linspace(0.02, 0.04, 50)
final_avg_fox_populations = plot_viability_threshold_new(f_growth_values, num_repeats, progress_bar, convergence_threshold=0.0)


### Plot MP4 animations

In [ ]:
from matplotlib import colormaps
from matplotlib.animation import FuncAnimation

# define colors for the three plots
BUNNY_CMAP = colormaps.get_cmap('Greens')
FOX_CMAP = colormaps.get_cmap('Reds')
WHITE = np.array([1.0, 1.0, 1.0])

def precompute_masks(n, scale):
    """
    Pre-computes the bunny and fox masks for the diagonal plot.
    
    args:
    n (int): The simulation grid size (e.g., 200).
    scale (int): The upscaling factor for pixels (e.g., 10).
    
    returns:
        mask_bunny (np.array): A 2D numpy array for masking bunny pixels.
    """
    print(f"Pre-computing {n*scale}x{n*scale} diagonal masks... This may take a moment.")
    
    # create a 2D array where px > py to create a diagonal layout
    px, py = np.indices((scale, scale))
    single_cell_mask = (px > py).astype(float) # 1.0 for fox, 0.0 for bunny

    # apply the diagonal layout to the entire grid (numpy.kron documentation)
    full_mask = np.kron(np.ones((n, n)), single_cell_mask)

    # indicator for fox or bunny
    mask_fox = full_mask
    mask_bunny = 1.0 - full_mask
    
    return mask_bunny, mask_fox

def create_diagonal_image_fast(bunnies, foxes, b_max, f_max, mask_bunny, mask_fox, scale=10):
    """
    Function to create the diagonal split image for the combined plot.

    args:
        bunnies (np.array): The n x n grid of bunny densities.
        foxes (np.array): The n x n grid of fox densities.
        b_max (float): The maximum bunny density for color normalization.
        f_max (float): The maximum fox density for color normalization.
        mask_bunny (np.array): The pre-computed bunny mask from precompute_masks.
        mask_fox (np.array): The pre-computed fox mask from precompute_masks.
        scale (int): The upscaling factor (must match the one used for masks).
    
    returns:
        diag_img (np.array): An (n*scale) x (n*scale) x 3 RGB image array.
    """
    # normalize populations
    b_norm = np.clip(bunnies / b_max, 0, 1)
    f_norm = np.clip(foxes / f_max, 0, 1)

    # get colors for density
    b_colors = BUNNY_CMAP(b_norm)[:, :, :3]
    f_colors = FOX_CMAP(f_norm)[:, :, :3]
    
    # if density=0 set to white
    b_colors[b_norm == 0] = WHITE
    f_colors[f_norm == 0] = WHITE

    # color the diagonal layout
    b_img_scaled = np.kron(b_colors, np.ones((scale, scale, 1)))
    f_img_scaled = np.kron(f_colors, np.ones((scale, scale, 1)))
    mask_b_3d = np.stack([mask_bunny]*3, axis=-1)
    mask_f_3d = np.stack([mask_fox]*3, axis=-1)
    
    # add the diagonal components
    diag_img = (b_img_scaled * mask_b_3d) + (f_img_scaled * mask_f_3d)
    
    return diag_img


def create_tri_plot_animation(sim, mask_bunny, mask_fox, scale=10, total_frames=300, steps_per_frame = 1, fps = 15, filename="tri_plot_simulation.mp4"):
    """
    Creates an mp4 animation with three plots:
    1. bunnies | 2. diagonal split | 3. foxes
    
    args:
        sim (BunnyFoxesSimulation): An initialized simulation object.
        mask_bunny (np.array): The pre-computed bunny mask from precompute_masks.
        mask_fox (np.array): The pre-computed fox mask from precompute_masks.
        scale (int): The upscaling factor (must match the one used for masks).
        total_frames (int): The total number of frames to render in the animation.
        steps_per_frame (int): The number of simulation steps to run between each frame.
        fps (int): The frames per second for the output video.
        filename (str): The output file name (e.g., "animation.mp4").

    returns:
        None
    """
    sim.initialize() # initialize simulation
    
    
    # setup the figure with 3 subplots
    fig, axes = plt.subplots(1, 3, figsize=(12, 5))
    ax_bunnies, ax_diag, ax_foxes = axes
    
    # get initial data
    bunnies = sim.bunnies
    foxes = sim.foxes
    # calculate the initial max fox value for consistent scaling
    fox_viz_max = np.max(foxes) + 1e-6
    if fox_viz_max < 5: fox_viz_max = 5
    
    # left plot: bunnies
    im_bunnies = ax_bunnies.imshow(bunnies, cmap='Greens', vmin=0, vmax=sim.b_cap)
    ax_bunnies.set_title('bunnies only')
    ax_bunnies.set_xticks([]); ax_bunnies.set_yticks([])
    
    # middle plot: diagonal split
    diag_img = create_diagonal_image_fast(bunnies, foxes, sim.b_cap, fox_viz_max, mask_bunny, mask_fox, scale)
    im_diag = ax_diag.imshow(diag_img)
    ax_diag.set_title('combined populations')
    ax_diag.set_xticks([]); ax_diag.set_yticks([])
    
    # right plot: foxes
    im_foxes = ax_foxes.imshow(foxes, cmap='Reds', vmin=0, vmax=fox_viz_max)
    ax_foxes.set_title('foxes only')
    ax_foxes.set_xticks([]); ax_foxes.set_yticks([])
    
    fig.suptitle('Step: 0', fontsize=16)
    plt.tight_layout(rect=[0, 0.03, 1, 0.95])

    # update the steps in the animation
    def animate(frame):
        
        # run simulation forward
        for _ in range(steps_per_frame):
            sim.update()
        
        # get new data
        bunnies = sim.bunnies
        foxes = sim.foxes
        # re-calculate the max fox value for this frame
        fox_viz_max = np.max(foxes) + 1e-6
        if fox_viz_max < 5: fox_viz_max = 5
        
        # create the new diagonal image
        diag_img = create_diagonal_image_fast(bunnies, foxes, sim.b_cap, fox_viz_max, mask_bunny, mask_fox, scale)
        
        # update all three plots
        im_bunnies.set_array(bunnies)
        im_diag.set_array(diag_img)
        im_foxes.set_array(foxes)
        im_foxes.set_clim(vmin=0, vmax=fox_viz_max) 
        
        # update title
        step_num = (frame + 1) * steps_per_frame # frame starts at 0
        fig.suptitle(f'Step: {step_num}', fontsize=16)
        
        print(f"\rRendering frame {frame+1}/{total_frames} (Sim Step {step_num})...", end="")
        
        return [im_bunnies, im_diag, im_foxes]

    anim = FuncAnimation(fig, animate, frames=total_frames, blit=True)
    anim.save(filename, writer='ffmpeg', fps=fps, dpi=100)
    print(f"\nSaved as{filename}'") 
    plt.close()


In [ ]:
# creating a simulation for the default parameters
MASK_SCALE = 10

sim_instance = BunnyFoxesSimulation(
        n=200, b_growth=0.3, b_diff=0.1,
        f_consump=0.1, f_growth=0.1, f_mort=0.03, f_diff=0.1
    )

# computing the masks
mask_b, mask_f = precompute_masks(sim_instance.n, MASK_SCALE)

# animation
create_tri_plot_animation(
    sim_instance, 
    mask_b, 
    mask_f, 
    scale=MASK_SCALE,
    filename="oscillating_simulation_INITIAL.mp4"
)

In [ ]:
# plotting snapshots of the simulation
def create_tri_plot_snapshots_separate(sim, snapshot_steps):
    """
    Creates a separate figure  for each snapshot step specified, 
    showing the same three plots: bunnies, diagonal split, foxes.
    
    args:
        sim (BunnyFoxesSimulation): an initialized simulation object.
        snapshot_steps (list): a list of integers specifying which steps to display.

    returns:
        None
    """
    sim.initialize()
    

    scale = 10
    bunny_cmap = colormaps.get_cmap('Greens')
    fox_cmap = colormaps.get_cmap('Reds')
    WHITE = (1.0, 1.0, 1.0)

    def create_diagonal_image(bunnies, foxes, b_max, f_max):
        # creates the diagonally split image.
        img = np.zeros((sim.n * scale, sim.n * scale, 3))
        for x in range(sim.n):
            for y in range(sim.n):
                b_norm = np.clip(bunnies[x, y] / b_max, 0, 1)
                f_norm = np.clip(foxes[x, y] / f_max, 0, 1)
                b_color = bunny_cmap(b_norm)[:3]; f_color = fox_cmap(f_norm)[:3]
                if b_norm == 0.0: b_color = WHITE
                if f_norm == 0.0: f_color = WHITE
                for px in range(scale):
                    for py in range(scale):
                        if px > py: img[x * scale + px, y * scale + py] = f_color
                        else: img[x * scale + px, y * scale + py] = b_color
        return img
    
    snapshot_steps = sorted(list(set([0] + snapshot_steps))) 
    num_snapshots = len(snapshot_steps)
    max_step_needed = snapshot_steps[-1]

    snapshot_idx = 0
    current_step = 0

    while current_step <= max_step_needed:
        # check if we need to capture this step
        if snapshot_idx < num_snapshots and current_step == snapshot_steps[snapshot_idx]:
            print(f"displaying snapshot at step {current_step}...")
            
            # create a new figure for this snapshot
            fig, axes = plt.subplots(1, 3, figsize=(12, 5))
            ax_bunnies, ax_diag, ax_foxes = axes
            
            bunnies = sim.bunnies
            foxes = sim.foxes
            fox_viz_max = np.max(foxes) + 1e-6
            if fox_viz_max < 5: fox_viz_max = 5
            
            # create the diagonal image
            diag_img = create_diagonal_image(bunnies, foxes, sim.b_cap, fox_viz_max)
            
            # plot the three images
            im_bunnies = ax_bunnies.imshow(bunnies, cmap='Greens', vmin=0, vmax=sim.b_cap)
            im_diag = ax_diag.imshow(diag_img)
            im_foxes = ax_foxes.imshow(foxes, cmap='Reds', vmin=0, vmax=fox_viz_max)

            # remove the ticks for better visibility
            for ax in [ax_bunnies, ax_diag, ax_foxes]:
                ax.set_xticks([])
                ax.set_yticks([])
            
            plt.tight_layout(rect=[0, 0, 1, 0.96]) # adjust rect for subtitle
            
            plt.show() 

            snapshot_idx += 1 # move to the next required snapshot

        # break if we have captured all snapshots
        if snapshot_idx >= num_snapshots:
             break
             
        # run one step of the simulation if we haven't reached the max step yet
        if current_step < max_step_needed:
             sim.update()
             current_step += 1
        else:
             break 

In [ ]:
# plotting sample snapshots
# base parameters
sim_instance = BunnyFoxesSimulation(
    n=200, b_growth=0.3, b_diff=0.3,
    f_consump=0.1, f_growth=0.1, f_mort=0.01, f_diff=0.1
)

# desired steps
snapshot_times = [1, 52, 71, 174] 
create_tri_plot_snapshots_separate(sim_instance, snapshot_steps=snapshot_times)

In [ ]:
def single_plot(sim, trials, plotting=True):
    '''
    Plots a single simulation for a simulation over a specified number of trials.
    
    args:
        sim (BunnyFoxesSimulation): an initialized simulation object.
        trials (int): number of steps (trials) to run the simulation for.
        plotting (bool): if True, plots results; if False, returns convergence status.

    returns:
        None (if plotting=True)
        1 or 0 (int, if plotting=False): 1 if converged, 0 if not.
    '''
    sim.initialize()
    
    b_density_history = np.zeros(trials)
    f_density_history = np.zeros(trials)
    
    for j in range(trials):
        b_density_history[j] = np.mean(sim.bunnies)
        f_density_history[j] = np.mean(sim.foxes)
        sim.update()

    interval_idx, avg_list, perc_list, conv_bool, _ = convergence_check(b_density_history, no_intervals=50)
    if plotting:
        plt.plot(b_density_history, label='bunnies', color='green', linewidth=1.5)
        plt.plot(f_density_history, label='foxes', color='sienna', linewidth=1.5)
        plt.legend()
        plt.grid(alpha=0.2)
        plt.show()
        print(conv_bool)
        if conv_bool:
            print(f"✔ The simulation has converged as the last three %-changes in mean are {perc_list[-4:]}")
        else:
            print(f"The simulation has NOT converged as the last three %-changes in mean are {perc_list[-4:]}")
    else:
        return 1 if conv_bool else 0

In [ ]:
# creating accompanying plot
sim_instance = BunnyFoxesSimulation(
        n=200, b_growth=0.3, b_diff=0.1,
        f_consump=0.1, f_growth=0.1, f_mort=0.03, f_diff=0.1
    )

# plotting vertical lines 
plt.vlines(1, -10, 170, color='red',alpha=0.8, linestyle="dotted")
plt.vlines(52, -10, 170, color='red',alpha=0.8, linestyle="dotted")
plt.vlines(71, -10, 170, color='red',alpha=0.8, linestyle="dotted")
plt.vlines(174, -10, 170, color='red',alpha=0.8, linestyle="dotted")
plt.text(5, 150, "①", color="red", fontsize=18)
plt.text(33, 150, "②", color="red", fontsize=18)
plt.text(75, 150, "③", color="red", fontsize=18)
plt.text(175, 150, "④", color="red", fontsize=18)
plt.ylim(-8, 165)
single_plot(sim_instance, 250)

### Pattern Analysis

In [ ]:
import time 
def plot_all_diffusion_patterns(base_params, b_diff_values, 
                                convergence_species='bunnies', 
                                num_trials=3, 
                                max_steps=10000, 
                                check_interval=200,
                                num_intervals_for_check=10,
                                convergence_threshold=0.01):
    """    
    Runs each simulation once and plots both the final bunny
    and fox 2D spatial patterns on two separate figures.

    args:
        base_params (dict): a dictionary of base simulation parameters.
        b_diff_values (list): a list of bunny diffusion values (d_b) to test.
        convergence_species (str): 'bunnies' or 'foxes', which population to check for convergence.
        num_trials (int): the number of repeat runs (rows) for each diffusion value.
        max_steps (int): maximum steps to run a simulation if it does not converge.
        check_interval (int): how often (in steps) to check for convergence.
        num_intervals_for_check (int): number of intervals to use in the convergence_check function.
        convergence_threshold (float): the percentage change threshold to define convergence.

    returns:
        None
    """
    
    if convergence_species not in ['bunnies', 'foxes']:
        print(f"warning: convergence_species must be 'bunnies' or 'foxes'. defaulting to 'bunnies'.")
        convergence_species = 'bunnies'
        
    num_ratios = len(b_diff_values)
    
    # create two subplot grids: one for bunnies, one for foxes
    fig_b, axes_b = plt.subplots(num_trials, num_ratios, 
                                 figsize=(num_ratios * 3.5, num_trials * 3.5),
                                 squeeze=False)
    fig_f, axes_f = plt.subplots(num_trials, num_ratios, 
                                 figsize=(num_ratios * 3.5, num_trials * 3.5),
                                 squeeze=False)
    
    print(f"starting parameter sweep for bunnies and foxes: {num_ratios} ratios, {num_trials} trials each.")
    print(f"using '{convergence_species}' for 'convergence_check' (interval={check_interval}, thresh={convergence_threshold}).")
    print("-" * 30)
    
    start_time = time.time()
    
    # loop over ratios (columns)
    for j, b_diff_val in enumerate(b_diff_values):
        
        f_diff_val = base_params.get('f_diff', 0.1)
        ratio = b_diff_val / f_diff_val
        print(f"running ratio {ratio:.2f} (d_b={b_diff_val}, d_f={f_diff_val})")

        # loop over trials (rows)
        for i in range(num_trials):
            
            # 1. setup simulation
            sim_params = base_params.copy()
            sim_params['b_diff'] = b_diff_val
            
            sim = BunnyFoxesSimulation(**sim_params)
            sim.initialize() 
            
            # get the correct subplot axis for each figure
            ax_b = axes_b[i, j] 
            ax_f = axes_f[i, j]
            
            # run simulation
            bunny_history = [] 
            fox_history = []
            converged = False
            step_count = 0
            status = f"max steps {max_steps}" # default
            avg_recent_abs_change = -1 # default
            
            while step_count < max_steps:
                sim.update()
                step_count += 1
                
                bunny_history.append(np.mean(sim.bunnies))
                fox_history.append(np.mean(sim.foxes))
                
                # check convergence
                min_steps_needed = num_intervals_for_check * 2 
                if step_count >= min_steps_needed and step_count % check_interval == 0:
                    
                    # check convergence based on the chosen species
                    if convergence_species == 'bunnies':
                        history_to_check = bunny_history
                    else:
                        history_to_check = fox_history
                        
                    _, _, _, conv_bool, avg_recent_abs_change = convergence_check(
                        history_to_check, 
                        no_intervals=num_intervals_for_check, 
                        threshold=convergence_threshold
                    )
                    
                    if conv_bool:
                        converged = True
                        status = f"converged @ {step_count}"
                        break # exit while loop
            
            print(f"  trial {i+1}: {status} (change: {avg_recent_abs_change:.4f}%)")

            # plot final state
            # plot bunnies
            ax_b.imshow(sim.bunnies, cmap='Greens', vmin=0, vmax=sim.b_cap)
            
            # plot foxes
            data_f = sim.foxes
            vmax_f = np.max(data_f) + 1e-6 
            if vmax_f < 1: vmax_f = 1
            ax_f.imshow(data_f, cmap='Reds', vmin=0, vmax=vmax_f)
            
            # add identical labels to both plots
            for ax in [ax_b, ax_f]:
                if i == 0: # top row
                    ax.set_title(f"ratio = {ratio:.2f}", fontsize=20)
                if i == num_trials - 1: # bottom row
                    ax.set_xlabel(f"d_b = {b_diff_val}", fontsize=20)
                if j == 0: # leftmost column
                    ax.set_ylabel(f"trial {i+1}", fontsize=20)
                    
                ax.set_xticks([])
                ax.set_yticks([])

    
    # finalize and show bunny plot
    fig_b.tight_layout()
    
    # finalize and show fox plot
    fig_f.tight_layout()
    
    plt.show() # shows both figures

In [ ]:
# base parameters
base_params = {
    'n': 200,
    'b_cap': 100.0,
    'b_growth': 0.3,
    'f_consump': 0.1,  
    'f_growth': 0.1,
    'f_mort': 0.03,
    'f_diff': 0.1
}

# test over the entire range
b_diff_values_to_test = [0.1,0.2,0.4,0.6,0.8]

# run
plot_all_diffusion_patterns(
    base_params=base_params, 
    b_diff_values=b_diff_values_to_test, 
    convergence_species='bunnies',
    num_trials=3, 
    max_steps=20000,
    check_interval=200,               
    num_intervals_for_check=10,       
    convergence_threshold=0.1        
)

# test over the zoomed in range
b_diff_values_to_test = [0.0, 0.02, 0.04, 0.06, 0.08, 0.1]

# run
plot_all_diffusion_patterns(
    base_params=base_params, 
    b_diff_values=b_diff_values_to_test, 
    convergence_species='bunnies',
    num_trials=3, 
    max_steps=20000,
    check_interval=200,               
    num_intervals_for_check=10,       
    convergence_threshold=0.1        
)

# test over high range
b_diff_values_to_test = [0.8,0.85,0.9,0.95,1]

# run
plot_all_diffusion_patterns(
    base_params=base_params, 
    b_diff_values=b_diff_values_to_test, 
    convergence_species='bunnies',
    num_trials=3, 
    max_steps=20000,
    check_interval=200,               
    num_intervals_for_check=10,       
    convergence_threshold=0.1        
)

### Analyze oscillations

In [ ]:
def plot_oscillation_time_series(base_params, b_growth_values_to_plot, 
                                 run_steps=5000):
    """
    Plots the full population time series for a few select bunny growth rates (g_b) 
    to visually check for oscillations.

    args:
        base_params (dict): a dictionary of base simulation parameters.
        b_growth_values_to_plot (list): a list of g_b values to simulate and plot.
        run_steps (int): the number of steps to run each simulation.

    returns:
        None
    """
    
    num_plots = len(b_growth_values_to_plot)
    
    print(f"running {num_plots} simulations for {run_steps} steps each...")
    
    # create a grid of plots, one for each g_b value stacked vertically
    fig, axes = plt.subplots(num_plots, 1, figsize=(10, num_plots * 3), sharex=True)
    
    # handle case of a single plot
    if num_plots == 1:
        axes = [axes]
    
    
    # loop over g_b values
    for i, b_growth_val in enumerate(b_growth_values_to_plot):
        
        ax = axes[i]
        print(f"  running g_b = {b_growth_val:.3f}...")
        for i in range(3):
            # setup simulation
            sim_params = base_params.copy()
            sim_params['b_growth'] = b_growth_val
            
            # use a different seed for each plot for robustness
            sim_params['seed'] = i 
            
            sim = BunnyFoxesSimulation(**sim_params)
            sim.initialize() 
            
            # storage for history
            bunny_history = np.zeros(run_steps)
            fox_history = np.zeros(run_steps)
            
            # run simulation for fixed steps
            for step in range(run_steps):
                sim.update()
                bunny_history[step] = np.mean(sim.bunnies)
                fox_history[step] = np.mean(sim.foxes)
                
            # plot the time series
            steps_axis = np.arange(run_steps)
            if i == 0: # add only one set of labels
                ax.plot(steps_axis, bunny_history, color='green', label='bunnies', alpha=0.7)
                ax.plot(steps_axis, fox_history, color='red', label='foxes', alpha=0.7)
            else:
                ax.plot(steps_axis, bunny_history, color='green', alpha=0.7)
                ax.plot(steps_axis, fox_history, color='red', alpha=0.7)
            ax.set_title(f"g_b = {b_growth_val:.3f}", fontsize=18)
            ax.set_ylabel("Mean Density", fontsize=18)
            ax.legend(fontsize=16)
            ax.grid(True, linestyle='--', alpha=0.6)

    # set shared x-axis label
    axes[-1].set_xlabel("time step")
    plt.tight_layout()
    plt.show()

In [ ]:
# plot oscillations
# base params
base_params = {
    'n': 200,
    'b_cap': 100.0,
    'b_diff': 0.1,
    'f_consump': 0.3,  
    'f_growth': 0.1,
    'f_mort': 0.03,
    'f_diff': 0.1      
}

# initial exploration of the entire range
values_to_plot = [0,0.1,0.2,0.3,0.4,0.5, 0.7, 0.9]
plot_oscillation_time_series(
    base_params=base_params, 
    b_growth_values_to_plot=values_to_plot, 
    run_steps=5000
)

# zoomed-in range
values_to_plot = [0.03,0.04,0.05,0.06,0.07,0.08,0.09,0.1]
plot_oscillation_time_series(
    base_params=base_params, 
    b_growth_values_to_plot=values_to_plot, 
    run_steps=5000
)


In [ ]:
# create animations to accompany the plots above
# for growth rate of 0.03
MASK_SCALE = 10

sim_instance = BunnyFoxesSimulation(
        n=200, b_growth=0.03, b_diff=0.1,
        f_consump=0.3, f_growth=0.1, f_mort=0.03, f_diff=0.1
    )

# computing the masks
mask_b, mask_f = precompute_masks(sim_instance.n, MASK_SCALE)

# animation
create_tri_plot_animation(
    sim_instance, 
    mask_b, 
    mask_f, 
    scale=MASK_SCALE,
    total_frames=200,
    steps_per_frame = 15, 
    fps = 15, 
    filename="growth_0.03.mp4"
)

In [ ]:
# for growth rate of 0.04
sim_instance = BunnyFoxesSimulation(
        n=200, b_growth=0.04, b_diff=0.1,
        f_consump=0.3, f_growth=0.1, f_mort=0.03, f_diff=0.1
    )

# computing the masks
mask_b, mask_f = precompute_masks(sim_instance.n, MASK_SCALE)

# animation
create_tri_plot_animation(
    sim_instance, 
    mask_b, 
    mask_f, 
    scale=MASK_SCALE,
    total_frames=200,
    steps_per_frame = 15, 
    fps = 15, 
    filename="growth_0.04.mp4"
)

In [ ]:
# for stable growth rate of 0.08
sim_instance = BunnyFoxesSimulation(
        n=200, b_growth=0.08, b_diff=0.1,
        f_consump=0.3, f_growth=0.1, f_mort=0.03, f_diff=0.1
    )

# computing the masks
mask_b, mask_f = precompute_masks(sim_instance.n, MASK_SCALE)

# animation
create_tri_plot_animation(
    sim_instance, 
    mask_b, 
    mask_f, 
    scale=MASK_SCALE,
    total_frames=200,
    steps_per_frame = 15, 
    fps = 15,
    filename="growth_0.08.mp4"
)

In [ ]:
from scipy.fft import rfft # import fft library

def get_dominant_frequency_amplitude(history):
    """
    Runs an FFT on the time series and returns the
    amplitude of the strongest non-zero frequency.

    args:
        history (np.array): a 1d numpy array of time series data.

    returns:
        dominant_amplitude (float): the amplitude of the strongest frequency (excluding f=0).
    """
    detrended_history = history - np.mean(history)
    
    # apply scipy.fft to get the power of each plot
    fft_amplitudes = rfft(detrended_history)
    fft_power = np.abs(fft_amplitudes)

    # discard low powers
    if len(fft_power) < 2:
        return 0 
    dominant_amplitude = np.max(fft_power[1:])
    return dominant_amplitude

def plot_oscillation_metric_with_errorbars(base_params, run_steps=5000, num_points=50, num_runs=5):
    """
    Runs simulations over a range of g_b values, averaging over
    num_runs for each point, and plots the FFT amplitude with error bars.

    args:
        base_params (dict): a dictionary of base simulation parameters.
        run_steps (int): the number of steps to run each individual simulation.
        num_points (int): the number of g_b values to test (points on the x-axis).
        num_runs (int): the number of repeat runs to average for each g_b value.
    
    returns:
        g_b_values (np.array): the array of bunny growth rate values tested.
        bunny_fft_mean (list): the list of mean bunny FFT amplitudes for each g_b value.
        fox_fft_mean (list): the list of mean fox FFT amplitudes for each g_b value.
    """
    
    # use 'zoomed in' growth values
    g_b_values = np.linspace(0, 0.15, num_points)
    
    # final data
    bunny_fft_mean = []
    bunny_fft_error = []
    fox_fft_mean = []
    fox_fft_error = []    
    base_seed = base_params.get('seed', 42)
    
    for g_b_val in g_b_values:
        if g_b_val % 0.05 < 0.01 or g_b_val == 0: # progress updates
             print(f"  Testing g_b = {g_b_val:.3f}...")
        
        # store the amplitudes
        run_bunny_amps = []
        run_fox_amps = []
        
        # run n times for each growth value
        for i in range(num_runs):
            sim_params = base_params.copy()
            sim_params['b_growth'] = g_b_val
            sim_params['seed'] = base_seed + i 
            # start simulation
            sim = BunnyFoxesSimulation(**sim_params)
            sim.initialize()
            
            bunny_history = np.zeros(run_steps)
            fox_history = np.zeros(run_steps)
            
            for step in range(run_steps):
                sim.update()
                bunny_history[step] = np.mean(sim.bunnies)
                fox_history[step] = np.mean(sim.foxes)
                
            # only analyse the last steps to exclude warm-up period
            transient_steps = run_steps // 2
            bunny_amp = get_dominant_frequency_amplitude(bunny_history[transient_steps:])
            fox_amp = get_dominant_frequency_amplitude(fox_history[transient_steps:])
            
            # store results
            run_bunny_amps.append(bunny_amp)
            run_fox_amps.append(fox_amp)
            
        # calculate descriptive state
        bunny_fft_mean.append(np.mean(run_bunny_amps))
        bunny_fft_error.append(np.std(run_bunny_amps))
        fox_fft_mean.append(np.mean(run_fox_amps))
        fox_fft_error.append(np.std(run_fox_amps))

    # plotting
    plt.figure(figsize=(10, 6))
    
    # mean with error bars
    plt.errorbar(g_b_values, bunny_fft_mean, yerr=bunny_fft_error, fmt='g-o', markersize=4,
                 label='Bunny Oscillation (FFT Power)', capsize=5, alpha=0.7)
    plt.errorbar(g_b_values, fox_fft_mean, yerr=fox_fft_error, fmt='r-o', markersize=4,
                 label='Fox Oscillation (FFT Power)', capsize=5, alpha=0.7)
    
    plt.title('Stability Analysis: Oscillation vs. Bunny Growth ($g_b$)')
    plt.xlabel('Bunny Growth Rate ($g_b$)')
    plt.ylabel('Dominant Frequency Amplitude (FFT)')
    plt.legend()
    plt.grid(True, linestyle='--', alpha=0.6)
    
    plt.yscale('log')
    
    # limit y axis for better visibility
    fox_fft_mean_np = np.array(fox_fft_mean)
    positive_fox_amps = fox_fft_mean_np[fox_fft_mean_np > 0]
    
    if len(positive_fox_amps) > 0:
        min_val = np.min(positive_fox_amps[np.nonzero(positive_fox_amps)]) / 10
        
    plt.ylim(bottom=10**(-10))
    plt.show()
    
    return g_b_values, bunny_fft_mean, fox_fft_mean

In [ ]:
pdf_params = {
    'n': 200, 'b_cap': 100.0, 'b_growth': 0.3, 'b_diff': 0.1, 
    'f_consump': 0.1, 'f_growth': 0.1, 'f_mort': 0.03, 'f_diff': 0.1,
    'p_bunny': 0.5, 'p_fox': 0.1, 'seed': 42
}

g_b_values, bunny_fft_mean, fox_fft_mean = plot_oscillation_metric_with_errorbars(
    pdf_params, 
    run_steps=10000,
    num_points=40,
    num_runs=5
)

### Theoretical Plots

In [ ]:
def calculate_b_star(d_f, g_f):
    """Calculates the stable bunny population."""
    return d_f / g_f

def calculate_f_star(g_b, c_f, k_b, b_star):
    """Calculates the stable fox population."""
    f_star = (g_b / c_f) * (1.0 - (b_star / k_b))
    if f_star < 0:
        return 0.0
    return f_star

# base parameters
params = {
    'g_b': 0.3,   
    'k_b': 100.0, 
    'g_f': 0.1,   
    'c_f': 0.3,   
    'd_f': 0.03   
}

# fox growth rate

g_f_values = np.linspace(0.05, 0.2, 100) 
b_star_results = []
f_star_results = []

for g_f_test in g_f_values:
    b_star = calculate_b_star(params['d_f'], g_f_test)
    f_star = calculate_f_star(params['g_b'], params['c_f'], params['k_b'], b_star)
    b_star_results.append(b_star)
    f_star_results.append(f_star)


# plot
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

# bunnies
ax1.plot(g_f_values, b_star_results, color='skyblue', linewidth=2)
ax1.set_xlabel('Fox Growth Rate ($g_f$)')
ax1.set_ylabel('Stable Bunny Pop. ($b^*$)')
ax1.set_title('Bunny Population vs. Fox Growth')
ax1.grid(True, linestyle=':')

# foxes
ax2.plot(g_f_values, f_star_results, color='salmon', linewidth=2)
ax2.set_xlabel('Fox Growth Rate ($g_f$)')
ax2.set_ylabel('Stable Fox Pop. ($f^*$)')
ax2.set_title('Fox Population vs. Fox Growth')
ax2.grid(True, linestyle=':')
plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()